**Mount directories and copy scripts**

In [1]:
# download data from google drive
# download conversion script

import shutil
from google.colab import drive

drive.mount('/content/drive')
shutil.copy("/content/drive/My Drive/Colab Notebooks/cifar_10_converter.py", "./")  

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'./cifar_10_converter.py'

**Prepare dataset**

In [0]:
import tensorflow as tf


DATA_DIR = 'data/cifar-10'

In [0]:
# convert original CIFAR-10 data to "usable" pickled format

import os
from cifar_10_converter import convert

if os.path.isdir(DATA_DIR):
  shutil.rmtree(DATA_DIR)
os.makedirs(DATA_DIR)
convert('/content/drive/My Drive/data/cifar-10/raw', DATA_DIR)

Test batch was converted
Train batch 1 of 5 was converted
Train batch 2 of 5 was converted
Train batch 3 of 5 was converted
Train batch 4 of 5 was converted
Train batch 5 of 5 was converted


**Define helpers**

In [0]:
# Wrappers


def get_weights(size, std, name): 
  weights = ...  # define variable, init with random values
  return weights


def get_biases(size, name):
  biases = ...  # define variable, init with zeros or you may change
                # function definition
  return biases


def add_conv_layer(tensor, k_size, stride, in_ch, out_ch, 
                   padding='SAME', act_fn=None, name=None):
  with tf.variable_scope(name):
    weights = get_weights(..... , ..., 'weights') # set propper arguments
    biases =  get_biases(...., 'biases') # set proper arguments

    outputs = tf.nn.conv2d(...)  # set proper arguments
    
    outputs = ... # add biases here
    outputs = ... # add activation fnction here
    
    return outputs


def add_pooling_layer(tensor, k_size, stride, padding='SAME', name=None):
  with tf.variable_scope(name):
    outputs = tf.nn.max_pool2d(...)  # set proper parameters here
    return outputs


def add_fc_layer(tensor, in_num, out_num, act_fn=None, name=None):
  with tf.variable_scope(name):
    weights = get_weights(..... , ..., 'weights') # set propper arguments
    biases =  get_biases(...., 'biases') # set proper arguments
    
    outputs = tf.matmul(...)  # set proper arguments
    outputs = tf.nn.bias_add(....) # set proper arguments
    outputs = ... # set proper arguments
    return outputs



**Define Training Routines**

In [0]:
def get_optimizer(lr_schedule, momentum):
  global_step = tf.Variable(0, 'global_step')
  
  learning_rate = tf.train.exponential_decay(
      lr_schedule['initial'],
      global_step,
      lr_schedule['step'],
      lr_schedule['decay'],
      staircase=True
  )
  
  # select one of the optimizer here:
  
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
#   optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
  
  # you may use Adam, AdaGrad
  return optimizer, global_step


**Define Network**

In [0]:
def lviv_net(tensor, is_training=None, name=None):
  af = tf.nn.relu
  name = name if name is not None else 'lviv_net'
  
  with tf.variable_scope(name):
    # define networktopology here
    # please use defined wrappers
      
    return tensor

  
def get_cls_loss(labels, logits, name=""):
  with tf.variable_scope(name):
    loss = ...  # define loss here
  return loss

def add_weights_decay(loss, w_decay, name=""):
  with tf.variable_scope(name):
    weights = tf.get_collection('WEIGHTS')  # do not forget to add weights in the collection in your wrapper
    loss += ...  # add weights L2 regularization
  return loss

def get_acc(labels, logits, name=""):
  
  acc = ...  # calculate accuracy here
  return acc

**Training iteslf**

In [0]:
import os
import time
from cifar_10_converter import unpickle


# read dataset
ds_train = unpickle(os.path.join(DATA_DIR, 'train.pkl'))
ds_test = unpickle(os.path.join(DATA_DIR, 'test.pkl'))

#setup optimization parameters
test_num = len(ds_test['images'])
images_num = len(ds_train['images'])
IMAGE_SIZE = 32
SCALE_TO = 32
INPUT_SIZE = 32
CLASSES_NUM = 10
lr_schedule = {
    'initial': 0.01,
    'step': 100 * images_num,
    'decay': 0.1
}
weights_decay = 0.0005
momentum = 0.9
batch_size = 250
epoch_num = 200
#---

tf.reset_default_graph()  # will allow you to restart the cell without problem
tf.set_random_seed(27)    # set random seed

optimizer, global_step = get_optimizer(lr_schedule, momentum)


# Create Model
feeded_img = tf.placeholder(tf.float32, shape=(batch_size, IMAGE_SIZE, IMAGE_SIZE, 3), name='feeded_img')
input_lbl = tf.placeholder(tf.int32, shape=(batch_size,), name='input_lbl')
is_training = tf.placeholder(tf.bool, shape=(), name='is_training')

# Do augmentation techniques
# at least:
#  - random crop
#  - resize
# you may also try brightness, contrast, etc.
  
  
# exctract mean pixel
# scale to [-1, 1]
input_img -= 128.
input_img /= 128.

logits = lviv_net(input_img, is_training=is_training)

loss = get_cls_loss(input_lbl, logits, name="loss")
add_weights_decay(loss, weights_decay, name="wdecay")

acc = get_acc(input_lbl, logits, name="accuracy")

# Create Training routines
train_op = optimizer.minimize(
    loss=loss,
    global_step=global_step
)

#  init variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
  # Run the initializer
  sess.run(init)
  max_acc = 0.
  # training loop
  for epoch in range(epoch_num):
    for i in range(0, images_num, batch_size):
      batch_x = ds_train['images'][i:i+batch_size]
      batch_y = ds_train['labels'][i:i+batch_size]
      
      # do training step here
      _, loss_value, acc_value = sess.run(
          [.....],   # set proper arguments here
          feed_dict={
              feeded_img: ....,   # set proper arguments here
              input_lbl: ....,   # set proper arguments here
              is_training: True
          }
      )
      
      if i % images_num == 0:
        test_loss = 0.
        test_acc = 0.
        for j in range(0, test_num, batch_size):
          test_x = ds_test['images'][j:j+batch_size]
          test_y = ds_test['labels'][j:j+batch_size]
          test_loss_value, test_acc_value = sess.run(
              [...],    # set proper arguments here
              feed_dict={
                  feeded_img: ...,     # set proper arguments here
                  input_lbl: ...,      # set proper arguments here
                  is_training: False
              }
          )
          test_loss += test_loss_value
          test_acc += test_acc_value
        test_loss /= (test_num / batch_size)
        test_acc /= (test_num / batch_size)
        if max_acc < test_acc:
          max_acc = test_acc
        print('epoch {} done. Accuracy={:.2f}/{:.2f} ({:.2f})'
              ' Loss={:.3f}/{:.3f}'
              .format(epoch+1, acc_value*100, test_acc*100, max_acc*100, 
                      loss_value, test_loss))
        
        
